In [1]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, get_constant_schedule_with_warmup, AutoModelForCausalLM, GenerationConfig, BitsAndBytesConfig,TextStreamer
from peft import (
    get_peft_model,
    LoraConfig,
prepare_model_for_kbit_training)
from utils import *
import copy
import json

[2024-07-26 05:50:51,268] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.1
 [WARNING]  using untested triton version (2.1.0), only 1.0.0 is known to be compatible


Error.  nthreads cannot be larger than environment variable "NUMEXPR_MAX_THREADS" (64)/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


# gpu 지정

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# quantization 

In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16)

# model 위치

In [2]:
model_path = "Qwen/Qwen2-72B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, quantization_config=bnb_config, device_map='auto', attn_implementation="flash_attention_2", torch_dtype=torch.bfloat16)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# prompt

In [3]:
def get_prompt(tokenizer, document, tokenize=False):
    user_message = f'''아래는 웹 페이지에서 발췌한 내용입니다. 
해당 페이지가 교육적 가치가 높은지 평가합니다.
초등학교부터 고등학교까지의 교육 환경에서 유용할 수 있는지 평가합니다.
아래에 설명된 5점 가산점 시스템을 사용하여 초등학교부터 고등학교 수준까지 교육용으로 유용할 수 있는지 평가합니다. 

점수는 각 기준의 만족도에 따라 누적됩니다:
- 광고 및 홍보 자료와 같이 교육 주제와 관련이 없거나 학술적이지 않은 내용이 일부 포함되어 있더라도 추출물이 교육 주제와 관련된 기본 정보를 제공하는 경우 1점을 추가합니다. 홍보 자료와 같은 비학술적 콘텐츠를 포함하더라도 1점을 추가합니다.
- 추출물이 교육과 관련된 특정 요소를 다루고 있지만 다음과 같은 경우 1점을 추가합니다. 교육 표준과 밀접하게 일치하지 않습니다. 교육용 콘텐츠와 잠재적으로 유용한 주제에 대한 피상적인 개요를 제공하는 비교육적 자료 또는 정보를 무질서한 방식으로 일관성 없는 글쓰기 스타일로 제시합니다.
- 발췌문이 교육용으로 적합하고 학교 커리큘럼과 관련된 주요 개념을 소개하는 경우 3점을 부여합니다. 학교 커리큘럼과 관련된 개념을 소개합니다. 포괄적이지는 않지만 일관성이 있거나 또는 일부 불필요한 정보를 포함할 수 있습니다. 교과서나 기본 튜토리얼의 소개 섹션과 비슷할 수 있습니다. 교과서 또는 학습에 적합하지만 다음과 같은 주목할 만한 한계가 있는 기본 튜토리얼의 소개 섹션과 유사할 수 있습니다. 초등학생에게 너무 복잡한 개념을 다루고 있습니다.
- 추출물이 교육적 목적과 관련성이 높고 유익한 경우 4점을 부여합니다. 초등학교 이하의 수준에서 명확하고 일관된 글쓰기 스타일을 보여주는 경우 4점을 부여합니다. 그것 교과서나 튜토리얼의 한 장과 유사하여 상당한 교육적 효과를 제공할 수 있습니다. 연습 문제와 해결책을 포함한 콘텐츠, 관련 없는 정보를 최소화하고 개념이 초등학생에게 너무 어렵지 않습니다. 콘텐츠가 일관성 있고 집중적이며 체계적인 학습에 도움이 됩니다.
- 추출물의 교육적 가치가 뛰어나고 다음에 완벽하게 적합한 경우 5점을 부여합니다. 초등학교 또는 중학교에서 가르치는 데 완벽하게 적합합니다. 자세한 추론을 따르고, 글쓰기 스타일이 따라 하기 쉽고 주제에 대한 심오하고 철저한 통찰력을 제공합니다, 비교육적이거나 복잡한 내용이 전혀 없습니다.
발췌문: <예시>.
발췌문을 검토한 후
- 다음 형식을 사용하여 점수로 마무리합니다: "교육적 점수: <총점>"
- 총 점수를 최대 3 문장 이내로 간략하게 정당화합니다.

다음은 발췌문입니다. {document}'''
    messages = [
            {"role": "user", "content": user_message}
            ]
    prompt = tokenizer.apply_chat_template(
            messages, 
            tokenize=tokenize, 
            add_generation_prompt=True
            )
    return prompt

# data load

In [5]:
data_path = '/home/work/user/ocw/data_filtering/2024_000.jsonl'
data = load_jsonl(data_path)

79778it [00:02, 35520.30it/s]


In [41]:
streamer = TextStreamer(tokenizer, skip_prompt=False, skip_special_tokens=True) 

# output dir

In [34]:
output_dir ='./qwen2/2024_000' 
os.makedirs(output_dir, exist_ok = True)

# output file name

In [2]:
file_name = 'ok.jsonl'
fout = open(os.path.join(output_dir, file_name), "w", encoding='utf-8')
for i in tqdm(data):
    document = i['text'] # input text
    prompt = get_prompt(tokenizer, document)
    inputs = tokenizer(prompt, return_tensors='pt', add_special_tokens=False)
    inputs = inputs.to(model.device)
    length = inputs['input_ids'].size(1)
    outputs = model.generate(**inputs, max_new_tokens=512, streamer=streamer, pad_token_id=tokenizer.eos_token_id)
    answer = tokenizer.decode(outputs[0][length:], skip_special_tokens=True)
    tmp = copy.deepcopy(i)
    tmp['response']=answer
    fout.write(json.dumps(tmp, ensure_ascii=False)+"\n")

NameError: name 'output_dir' is not defined